# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dzhetygara,52.1908,61.2006,-14.49,96,84,3.55,KZ,1706653403
1,1,kristiansund,63.1115,7.7320,2.36,44,67,1.54,NO,1706653403
2,2,assinovskaya,43.2400,45.1800,-2.76,93,16,1.48,RU,1706653404
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.37,60,63,4.20,SH,1706653404
4,4,vadso,70.0744,29.7487,4.09,70,0,11.32,NO,1706653404


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points(x = "Lng", y = "Lat", size = "Humidity", tiles = True, color = "Humidity", xlabel = "Longitude", ylabel = "Latitude") #Reference 1

# Display the map
humid_map

c:\Users\cpord\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df["Max Temp"] >21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,port elizabeth,-33.9180,25.5701,22.40,85,0,0.51,ZA,1706653406
187,187,williamson,30.6502,-97.6003,24.42,31,0,2.24,US,1706653439
241,241,mermoz boabab,14.7065,-17.4758,22.88,73,0,3.09,SN,1706653453
441,441,lompoc,34.6391,-120.4579,24.27,57,0,2.57,US,1706653499
467,467,inhambane,-23.8650,35.3833,24.26,82,0,3.87,MZ,1706653523


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,port elizabeth,ZA,-33.9180,25.5701,85,None
187,williamson,US,30.6502,-97.6003,31,None
241,mermoz boabab,SN,14.7065,-17.4758,73,None
441,lompoc,US,34.6391,-120.4579,57,None
467,inhambane,MZ,-23.8650,35.3833,82,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius" : radius,
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row[3]
    lat = row[2]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
williamson - nearest hotel: Holiday Inn Express & Suites Georgetown
mermoz boabab - nearest hotel: La maison blanche
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
inhambane - nearest hotel: Hotel de Inhambane


,City,Country,Lat,Lng,Humidity,Hotel Name
14,port elizabeth,ZA,-33.9180,25.5701,85,Waterford Hotel
187,williamson,US,30.6502,-97.6003,31,Holiday Inn Express & Suites Georgetown
241,mermoz boabab,SN,14.7065,-17.4758,73,La maison blanche
441,lompoc,US,34.6391,-120.4579,57,Embassy Suites by Hilton Lompoc Central Coast
467,inhambane,MZ,-23.8650,35.3833,82,Hotel de Inhambane


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(x = "Lng", y = "Lat", tiles = True, size  = 100, hover_cols = ["Hotel Name", "Country"], xlabel = "Longitude", ylabel = "Latitude", color = "Humidity") #Reference 1

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)